In [1]:
%%time
import pandas as pd
import numpy as np
import json

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from scipy.sparse import hstack

import matplotlib.pyplot as plt
np.random.seed(0)

Wall time: 23.9 s


In [2]:
import os
print(os.listdir("../input"))

['News_Category_Dataset.json']


In [3]:
n_df = pd.read_json('../input/News_Category_Dataset.json', lines=True)

In [4]:
n_df['hds']=n_df['headline']+n_df['short_description']

In [5]:
import nltk
import string
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
stemmer = PorterStemmer()

In [6]:
def text_process(mess):
    nopunc=[char for char in mess if char not in string.punctuation]
    new1=''.join(nopunc)
    new2=[stemmer.stem(word) for word in new1]
    new3=''.join(new2)
    return[word for word in new3.split()if word.lower()not in stopwords.words('english') ]

In [7]:
n_df['hds'].head()

0    There Were 2 Mass Shootings In Texas Last Week...
1    Will Smith Joins Diplo And Nicky Jam For The 2...
2    Hugh Grant Marries For The First Time At Age 5...
3    Jim Carrey Blasts 'Castrato' Adam Schiff And D...
4    Julianna Margulies Uses Donald Trump Poop Bags...
Name: hds, dtype: object

In [8]:
n_df['hds'].head(5).apply(text_process)

0    [2, Mass, Shootings, Texas, Last, Week, 1, TVS...
1    [Smith, Joins, Diplo, Nicky, Jam, 2018, World,...
2    [Hugh, Grant, Marries, First, Time, Age, 57The...
3    [Jim, Carrey, Blasts, Castrato, Adam, Schiff, ...
4    [Julianna, Margulies, Uses, Donald, Trump, Poo...
Name: hds, dtype: object

In [9]:
train_text, test_text, ytrain, ytest = train_test_split(
    n_df['hds'], n_df['category'], random_state=52)

In [10]:
%%time
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    ngram_range=(1, 10))
word_vectorizer.fit(train_text)

CPU times: user 2min 25s, sys: 8.82 s, total: 2min 33s
Wall time: 2min 33s


In [11]:
%%time
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(1, 5))
char_vectorizer.fit(train_text)

CPU times: user 1min 5s, sys: 2.58 s, total: 1min 7s
Wall time: 1min 7s


In [12]:
%%time
X = hstack([word_vectorizer.transform(train_text), char_vectorizer.transform(train_text)])

CPU times: user 1min 50s, sys: 5.12 s, total: 1min 55s
Wall time: 1min 55s


In [13]:
from sklearn.linear_model import SGDClassifier
sgd_cls = SGDClassifier(max_iter=20)
sgd_cls.fit(X, ytrain)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=20,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [14]:
predict = sgd_cls.predict(hstack([word_vectorizer.transform(test_text), char_vectorizer.transform(test_text)]))

In [15]:
acc = np.mean(ytest == predict)

In [16]:
print('accuracy: {0:.3}'.format(acc))

accuracy: 0.601
